<a href="https://colab.research.google.com/github/Namesakenberg/Deep_Learning/blob/main/complete_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [73]:
df = pd.read_csv('/content/diabetes.csv')

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [75]:
X = df.drop(columns = ['Outcome'])
y = df['Outcome']

In [76]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X, y , test_size=0.2 , random_state = 42)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

In [77]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [78]:
pip install keras-tuner --upgrade

In [79]:
import keras_tuner as kt

In [80]:
def make_model(hp):
  model1 = Sequential()
  for i in range(hp.Int('num_layers',min_value = 1 , max_value = 10)):   # number of layers to have
    if i==0 :
      model1.add(Dense(
          hp.Int((f'num_neurons_{i}'),min_value = 8 , max_value = 128) ,
          activation = hp.Choice(f'activation_{i}',values=['tanh' , 'sigmoid' , 'relu']),
          input_dim=X_train_scaled.shape[1]))
      model1.add(Dropout(hp.Choice('dropout' ,values = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else:
      model1.add(Dense(
        hp.Int((f'num_neurons_{i}'),min_value=8,
                max_value = 128 ) ,
              activation=hp.Choice(f'activation_{i}',values = ['tanh' , 'sigmoid' ,'relu'])))
  model1.add(Dense(1 , activation='sigmoid'))
  model1.compile(optimizer = hp.Choice('optimizer' ,values =['adam' ,'rmsprop','Adagrad','sgd']) ,loss = 'binary_crossentropy',metrics=['accuracy'])
  return model1



In [81]:
tuner = kt.RandomSearch(make_model , objective='val_accuracy' , max_trials=5 , overwrite=True)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [82]:
tuner.search(X_train_scaled , y_train , epochs = 5 , validation_data =(X_test_scaled , y_test))

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6818181872367859
Total elapsed time: 00h 00m 28s


In [83]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'num_neurons_0': 58,
 'activation_0': 'tanh',
 'dropout': 0.2,
 'optimizer': 'Adagrad',
 'num_neurons_1': 120,
 'activation_1': 'tanh',
 'num_neurons_2': 125,
 'activation_2': 'relu',
 'num_neurons_3': 8,
 'activation_3': 'tanh',
 'num_neurons_4': 8,
 'activation_4': 'tanh',
 'num_neurons_5': 8,
 'activation_5': 'tanh'}

In [84]:
model1= tuner.get_best_models(num_models = 1)[0]
model1.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 2 variables whereas the saved optimizer has 16 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 58)             │           522 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 58)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 120)            │         7,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 125)            │        15,125 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │         1,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 8)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,888 (93.31 KB)

 Trainable params: 23,888 (93.31 KB)

 Non-trainable params: 0 (0.00 B)

In [85]:
model1.fit(X_train_scaled , y_train , epochs = 100 , initial_epoch=6 , validation_data=(X_test_scaled,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.6323 - loss: 0.6631 - val_accuracy: 0.7143 - val_loss: 0.6417
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.6436 - loss: 0.6573 - val_accuracy: 0.7143 - val_loss: 0.6322
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6896 - loss: 0.6397 - val_accuracy: 0.7273 - val_loss: 0.6243
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7000 - loss: 0.6356 - val_accuracy: 0.7338 - val_loss: 0.6175
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7107 - loss: 0.6304 - val_accuracy: 0.7273 - val_loss: 0.6116
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7533 - loss: 0.6100 - val_accuracy: 0.7208 - val_loss: 0.6064
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7035 - loss: 0.6194 - val_accuracy: 0.7273 - val_loss: 0.6021
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.7003 - loss: 0.6242 - val_accurac